# File Variables to Use #

In [1]:
!ls

DataExplorationCont.ipynb
DataInvestigation_Alex.ipynb
InitialDataExploration.ipynb
Movie-Data-Project1.ipynb
Neptune.ipynb
README.md
Untitled.ipynb
zippedData


In [126]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.5f}'.format

movie_gross_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz') # CLEANED except for year/title thing
imdb_name_basics_df = pd.read_csv('zippedData/imdb.name.basics.csv.gz')
imdb_title_akas_df = pd.read_csv('zippedData/imdb.title.akas.csv.gz')
imdb_title_basics_df = pd.read_csv('zippedData/imdb.title.basics.csv.gz') # JENNIFER 
imdb_title_crew_df = pd.read_csv('zippedData/imdb.title.crew.csv.gz')
imdb_title_principals_df = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
imdb_title_ratings_df = pd.read_csv('zippedData/imdb.title.ratings.csv.gz') #ALEX 
# rt_movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t')
# rt_reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter = '\t', encoding='latin1')
tmdb_movies_df = pd.read_csv('zippedData/tmdb.movies.csv.gz') #CLEAN 
tn_movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz') #PATRICK 

## Cleaning To Dos for Movie Gross ##
Get rid of rows with null values in the studio field and in the domestic gross field since there aren't many of them
ignore the foreign gross nulls, info may be listed on other sheets iirc. Alternately possibly could be calculated w/ info from other sheets
do a unique value check on year, make sure nothing weird
do a unique value check on studio, can we replace w/proper names?
sort by domestic gross and check the tail
clean out "(YEAR)" from movie titles (via regular expression?)

In [14]:
movie_gross_df.dropna(subset=['studio','domestic_gross'], inplace = True)


In [15]:
#importing regex so I can more easily find the films with the year added to their titles

import re

In [16]:

# df[df['Country (region)'].str.count('^[pP].*')>0]
#movie_gross_df[movie_gross_df['title'].str.count("/\(([^()\]*)\)/g'")>0]

## Cleaning To Dos for tmdb_movies_df ##
- add genre columns
- check min/maxs
- note no nulls
- clean out rows w/empty lists in genre category (like 10% of the data set, 2.6k, more then I like but average isn't going to cut it and there's no way to manually enter that many)

In [17]:
# Gets rid of row of empty lists in google

tmdb_movies_df.drop(tmdb_movies_df[tmdb_movies_df['genre_ids'] == '[]'].index, inplace = True)

In [18]:
## going to try and create a column for each genre based on the list and then populate it with boolean for the particular film
## start going to make a dictionary of the codes, then going to loop through, each loop creates a column and populates it

genre_dict = {28:'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80:'Crime', 99:'Documentary',18:'Drama', 10751:'Family',
              14:'Fantasy', 36: 'History', 27:'Horror', 10402:'Music', 9648:'Mystery', 10749:'Romance', 878:'Science Fiction',
              53:'Thriller', 10752:'War',37:'Western'}
#didn't include TV Movie category since we don't care about those (may want to filter them out...)

In [19]:
# clears out the tv movies 
tmdb_movies_df.drop(tmdb_movies_df.loc[tmdb_movies_df['genre_ids'].str.contains('10770')].index, inplace = True)

In [20]:
## creates new columns with True/False values

for key, value in genre_dict.items():
    tmdb_movies_df[value] = tmdb_movies_df['genre_ids'].str.contains(str(key))

## QUESTIONS FOR TEACHERS ##

- WHy is the GitHub notebook so different from the "live" version?
- How to get regular expression to work? 
- What kind of graph are good to show a category with multiple values aka genre - stacked bar chart?

## INVESTIGATION GOALS ##

- Alex: genre by profit or genre by likes, if possible some kind of stacked bar chart, What kinds of films are popular? 
- Patrick: production budget v profit - gross vs foreign, possibly incorporating genre, What films are the most profitable? 
- Jennifer: genre over **time** - aka What's being made? What studio is making what?

Checking out which columns have problamatic null values.

In [22]:
imdb_title_basics_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


We decided to keep the rows with null values in runtime_minutes just in case the column is useful later. I will replace the NaN with zeros. If need be we can change them later to something more relevant.

In [34]:
imdb_title_basics_df['runtime_minutes'] = imdb_title_basics_df['runtime_minutes'].replace(np.nan, 0)

<ipython-input-34-b423d9fb8552>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_title_basics_df['runtime_minutes'] = imdb_title_basics_df['runtime_minutes'].replace(np.nan, 0)


In [35]:
imdb_title_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           140736 non-null  object 
 1   primary_title    140736 non-null  object 
 2   original_title   140736 non-null  object 
 3   start_year       140736 non-null  int64  
 4   runtime_minutes  140736 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.5+ MB


I decided to drop the rows in genres with NaN because the amount of rows was a small percentage of the overall dataset. 

In [36]:
imdb_title_basics_df = imdb_title_basics_df.dropna(subset = ['genres'])

I will replace the null values in original_title to 0. There are so few I don't expect it to be an issues. We also may end up not needing this column.

In [37]:
imdb_title_basics_df['original_title'] = imdb_title_basics_df['original_title'].replace(np.nan, 0)

In [38]:
imdb_title_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           140736 non-null  object 
 1   primary_title    140736 non-null  object 
 2   original_title   140736 non-null  object 
 3   start_year       140736 non-null  int64  
 4   runtime_minutes  140736 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.5+ MB


I need to make a dictionary to compare genre values to in order to split them up

In [103]:
genre_list = imdb_title_basics_df['genres'].unique().tolist()

In [104]:
genre_list_2 = []
genre_list_3 = []
for x in genre_list:
    genre_list_2.append(x.split(sep = ','))
    

In [105]:
for x in genre_list_2:
    for y in x:
        genre_list_3.append(y)

In [106]:
genre_dict = pd.unique(genre_list_3)

In [107]:
genre_dict = genre_dict.tolist()

In [108]:
genre_dict

['Action',
 'Crime',
 'Drama',
 'Biography',
 'Comedy',
 'Fantasy',
 'Horror',
 'Thriller',
 'Adventure',
 'Animation',
 'Documentary',
 'History',
 'Mystery',
 'Sci-Fi',
 'Romance',
 'Family',
 'War',
 'Music',
 'Sport',
 'Western',
 'Musical',
 'Adult',
 'News']

In [109]:
len(genre_dict)

23

I'm going to drop any row that has values matching Talk-Show, Reality-TV, Game-Show, and Short. There are not many rows with these values and they are not applicaple to our analysis. 

In [97]:
short_drop = imdb_title_basics_df[imdb_title_basics_df['genres'].str.contains('Short')].index
game_show_drop = imdb_title_basics_df[imdb_title_basics_df['genres'].str.contains('Game-Show')].index
talk_show_drop = imdb_title_basics_df[imdb_title_basics_df['genres'].str.contains('Talk-Show')].index
reality_tv_drop = imdb_title_basics_df[imdb_title_basics_df['genres'].str.contains('Reality-TV')].index


In [101]:
imdb_title_basics_df.drop(short_drop, inplace = True)
imdb_title_basics_df.drop(game_show_drop, inplace = True)
imdb_title_basics_df.drop(talk_show_drop, inplace = True)
imdb_title_basics_df.drop(reality_tv_drop, inplace = True)


In [133]:
for x in genre_dict:
    imdb_title_basics_df[x] = imdb_title_basics_df['genres'].str.contains(str(x))

In [134]:
imdb_title_basics_df.rename(columns = {'Sci-Fi':'Science Fiction' }, inplace = True)

In [135]:
imdb_title_basics_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,Action,Crime,Drama,Biography,...,Science Fiction,Romance,Family,War,Music,Sport,Western,Musical,Adult,News
0,tt0063540,Sunghursh,Sunghursh,2013,175.00000,"Action,Crime,Drama",True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.00000,"Biography,Drama",False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.00000,Drama,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,nan,"Comedy,Drama",False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.00000,"Comedy,Drama,Fantasy",False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
